### Property-based testing

https://hypothesis.readthedocs.io/en/latest/

From the Hypothesis docs:

It works by letting you write tests that assert that something should be true for every case, not just the ones you happen to think of.

Think of a normal unit test as being something like the following:

    1. Set up some data.
    2. Perform some operations on the data.
    3. Assert something about the result.

Hypothesis lets you write tests which instead look like this:

    1. For all data matching some specification.
    2. Perform some operations on the data.
    3. Assert something about the result.
    
This is often called property based testing, and was popularised by the Haskell library Quickcheck.

In [ ]:
import json
import re
from collections import Counter

import pytest

from hypothesis import given
import hypothesis.strategies as st

* A test in Hypothesis consists of two parts: 
    1. A function that looks like a normal test in your test framework of choice but with some additional arguments
    2. `@given` decorator that specifies how to provide those arguments.

https://hypothesis.readthedocs.io/en/latest/data.html?highlight=regex#hypothesis.strategies.from_regex

In [ ]:
# st.from_regex(regex, *, fullmatch=False)

regex = r'^[A-Z]\w+$'

@given(st.from_regex(regex=regex, fullmatch=False))
def test_print_text(text):
    print(text)

test_print_text()

In [ ]:
# * means any number of times
regex = r'\D*(\d{3})\D*(\d{3})\D*(\d{4})\D*$'
regex = r'^\d\Z'

@given(st.from_regex(regex=regex, fullmatch=False))
def test_print_num(num):
    print(repr(num), ord(num))
    
test_print_num()                   

* Normalize phone number

In [ ]:
# do not delete!
phones = [
   "(713) 439-6000",
    "7134396000",
    "713-4396000",
    "713-439-6000",
    " 713-439-6000",
    "713-439-6000",
]

In [ ]:
PHONE_TEST_CASES = {  # phone_number -> "###-###-####"
    "(713) 555-6000": "713-555-6000",
    "7135556000": "713-555-6000",
    "713-5556000": "713-555-6000",
    "713-555-6000": "713-555-6000",
    " 713-555-6000": "713-555-6000",
    "713-555-6000 ": "713-555-6000",
    "17135556000": "ERR-BAD-NMBR",
    "": "999-999-9999",
    None: "999-999-9999",
}

In [ ]:
def test_normalize_phone_number(parser):
    for phone_number in PHONE_TEST_CASES:
        expected = PHONE_TEST_CASES[phone_number]
        res = parser(phone_number)
        if res == expected:
            print(f"PASS: {phone_number!r} -> {res!r}")
        else:
            print(f"FAIL: {phone_number!r} -> {res!r}")

In [ ]:
# test_normalize_phone_number(normalize_phone_number)

In [ ]:
PHONE_PAT = re.compile(r'^\D*(\d{3})\D*(\d{3})\D*(\d{4})\D*$')
PHONE_PAT = re.compile(r'\D*(\d{3})\D*(\d{3})\D*(\d{4})\D*$')

def normalize_phone_number(phone_number):  # provider_phone_number
    if not phone_number:
        return "999-999-9999"
    match = PHONE_PAT.match(phone_number)
    if not match:
        return "ERR-BAD-NMBR"
    # return match.groups()
    area_code, central_office_code, remainder = match.groups()
#    return area_code, central_office_code, remainder
    return f"{area_code}-{central_office_code}-{remainder}"

In [ ]:
test_normalize_phone_number(normalize_phone_number)

### Hypothesis

In [ ]:
# ZD: As an additional test, not replacement test: just check that it runs without errors!

@given(st.from_regex(PHONE_PAT))
def test_normalize_phone_number_no_error(parser, phone_number):
    parser(phone_number)

In [ ]:
test_normalize_phone_number_no_error(normalize_phone_number)